In [7]:

import utils.db_utils as db_utils
import utils.projection_utils as proj_utils
import utils.adsb_utils as adsb_utils
import pandas as pd
# reload import utils.db_utils
import importlib
importlib.reload(db_utils)

db = db_utils.ContrailDatabase('/Users/shrenikborad/pless/contrails/contrails_uwisc_2025-01-09.duckdb')
contrails_df = db.get_all_df()

In [8]:
date_str = "2025-01-09"
df = pd.read_csv("/Users/shrenikborad/Downloads/NNDL/adsb_flightpings_wisconsin_2025-01-09.csv")
from_dt = pd.to_datetime(f"{date_str} 06:00:00").tz_localize('America/Chicago').tz_convert('UTC')
to_dt = pd.to_datetime(f"{date_str} 19:00:00").tz_localize('America/Chicago').tz_convert('UTC')
df['time'] = pd.to_datetime(df['time'])
df = df[(df['time'] >= from_dt) & (df['time'] < to_dt)]
print(df.describe())
df_upsampled = adsb_utils.get_upsampled_df_for_day(df, max_range_m=50000)


               lon          lat           alt      alt_gnss      heading  \
count  5259.000000  5259.000000   5256.000000   5241.000000  5259.000000   
mean    -89.385266    43.100603   9706.844559   9572.500477   195.091082   
std       0.280539     0.194544  12373.197535  12269.843931    97.826588   
min     -90.017000    42.622000    600.000000    750.000000     0.000000   
25%     -89.591000    42.968000   2200.000000   2150.000000   112.000000   
50%     -89.377000    43.127000   3500.000000   3325.000000   196.000000   
75%     -89.183000    43.233000   9437.500000   9325.000000   276.000000   
max     -88.793000    43.518000  43000.000000  42675.000000   360.000000   

         alt_meters  alt_gnss_meters    distance_m  
count   5256.000000      5241.000000   5259.000000  
mean    2958.646221      2917.698145  28752.643432  
std     3771.350609      3739.848430  13269.798997  
min      182.880000       228.600000    770.763991  
25%      670.560000       655.320000  16410.341793

/Users/shrenikborad/pless/contrails/utils/adsb_utils.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])
/Users/shrenikborad/pless/contrails/utils/adsb_utils.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alt_gnss_meters'] = df['alt_gnss_meters'].astype(float)
/Users/shrenikborad/pless/contrails/utils/adsb_utils.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

Processed 100 aircraft...
Processed 110 aircraft...
Processed 120 aircraft...
Processed 130 aircraft...
Processed 140 aircraft...
Processed 150 aircraft...
Processed 160 aircraft...
Processed 170 aircraft...
Processed 180 aircraft...


/Users/shrenikborad/pless/contrails/utils/adsb_utils.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['isUpsampled'] = merged['isUpsampled'].fillna(True)
/Users/shrenikborad/pless/contrails/utils/adsb_utils.py:58: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged[col] = merged[col].fillna(method='ffill')
/Users/shrenikborad/pless/contrails/utils/adsb_utils.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['isUpsampled'] = merged['isUpsampled']

Processed 190 aircraft...
Processed 200 aircraft...
Processed 210 aircraft...
Processed 220 aircraft...
Processed 230 aircraft...
Processed 240 aircraft...
Processed 250 aircraft...


/Users/shrenikborad/pless/contrails/utils/adsb_utils.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['isUpsampled'] = merged['isUpsampled'].fillna(True)
/Users/shrenikborad/pless/contrails/utils/adsb_utils.py:58: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged[col] = merged[col].fillna(method='ffill')
/Users/shrenikborad/pless/contrails/utils/adsb_utils.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['isUpsampled'] = merged['isUpsampled']

In [ ]:
frames = []
labels = []
#  group by 10 seconds
for t, group in contrails_df.groupby(contrails_df['timestamp'].dt.floor('10S')):
    heat_data = group[['lat', 'lon']].values.tolist()
    frames.append(heat_data)
    labels.append(t.strftime("%Y-%m-%d %H:%M:%S"))


# show flights as points with identifiers
# keep only data with timestamp matching the contrail frames
print(labels[:5])
flight_frames = []
for t, group in df_upsampled.groupby(df_upsampled['time']):
    if t in labels:
        flight_data = group[['lat', 'lon', 'ident']].values.tolist()
        flight_frames.append(flight_data)

len(flight_frames)

/var/folders/nk/x5y68p8n5v529rrl_01kmfh80000gn/T/ipykernel_72329/1904896552.py:4: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  for t, group in contrails_df.groupby(contrails_df['timestamp'].dt.floor('10S')):


KeyError: 'timestamp'

In [2]:
# show folium with geojson with time slider

origin = [43.0710755318052, -89.40681496931353]
import folium
from folium.plugins import HeatMapWithTime
from folium.plugins import TimestampedGeoJson

# Create separate maps for each visualization
m1 = folium.Map(location=origin, zoom_start=5)
m2 = folium.Map(location=origin, zoom_start=5)
m_combined = folium.Map(location=origin, zoom_start=5)

# Map 1: Only flights with TimestampedGeoJson
features = []
for i, flight_data in enumerate(flight_frames):
    timestamp = labels[i] if i < len(labels) else labels[-1]
    for flight in flight_data:
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [flight[1], flight[0]]  # lon, lat
            },
            'properties': {
                'time': timestamp,
                'popup': flight[2],
                'icon': 'circle',
                'iconstyle': {
                    'fillColor': 'blue',
                    'fillOpacity': 0.6,
                    'stroke': 'true',
                    'radius': 2
                }
            }
        }
        features.append(feature)

TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='PT10S',
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=5,
    loop_button=True,
    date_options='YYYY-MM-DD HH:mm:ss',
    time_slider_drag_update=True,
    duration="P2M",
).add_to(m1)

# Map 2: Only contrails heatmap
HeatMapWithTime(frames,
    index=labels,
    auto_play=True,
    max_opacity=0.8,
    radius=8
).add_to(m2)

# Map 3: Combined approach - Use only HeatMapWithTime and add flight points as regular markers
# First add the heatmap
HeatMapWithTime(frames,
    index=labels,
    auto_play=True,
    max_opacity=0.8,
    radius=8
).add_to(m_combined)

# Add some sample flight points as static markers (since time sync is tricky)
sample_flights = df_upsampled.sample(min(100, len(df_upsampled)))
for _, flight in sample_flights.iterrows():
    folium.CircleMarker(
        location=[flight['lat'], flight['lon']],
        radius=3,
        popup=f"Flight: {flight['ident']}",
        color='blue',
        fill=True,
        fillColor='lightblue',
        fillOpacity=0.7
    ).add_to(m_combined)

print("Map 1: Flights only with time slider")
display(m1)
print("\nMap 2: Contrails heatmap only")
display(m2)
print("\nMap 3: Combined (heatmap + static flight markers)")
display(m_combined)

NameError: name 'flight_frames' is not defined